###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2018  by D. Koehn, heterogeneous models are from [this Jupyter notebook](https://nbviewer.jupyter.org/github/krischer/seismo_live/blob/master/notebooks/Computational%20Seismology/The Finite-Difference Method/fd_ac2d_heterogeneous.ipynb) by Heiner Igel ([@heinerigel](https://github.com/heinerigel)), Florian Wölfl and Lion Krischer ([@krischer](https://github.com/krischer)) which is a supplemenatry material to the book [Computational Seismology: A Practical Introduction](http://www.computational-seismology.org/), notebook style sheet by L.A. Barba, N.C. Clementi

###### Translated by Huizhe Di @ SCSIO

# 练习: Marmousi-2 模型的二维声学有限差分建模

在这个练习中，你必须应用所有关于FD建模的知识，到目前为止，我们已经涵盖了。虽然上一课的建模例子很简单，我们现在在一个更现实的问题中计算二维声波传播，称为 **Marmousi-2模型**.
这个模型由French Petroleum Institute (IFP)在1990s发展而来  ([Versteeg, 1994](https://library.seg.org/doi/abs/10.1190/1.1437051)), Marmousi模型是地震成像和反演技术中广泛使用的基准问题。除了模型原始的声学版本，弹性学的版本由[Martin et al. (2006)](https://library.seg.org/doi/abs/10.1190/1.2172306)发展而来.

Marmousi-2模型包含了一个 460 m 厚的水层在弹性地下模型之上。沉积模型在左右边界附近非常简单，而在中心区域则相当复杂。在两边，海底几乎是水平分层的，而陡峭的逆冲断层扰乱了模型中心的层。在逆冲断层系统和地层中埋藏着小型油气藏。

##### 练习

在Marmousi-2模型中建立二维声波传播模型:

- 根据给定的Marmousi-2 p波速度模型定义模型离散化。最大波传播时间应为6秒
- 计算地震子波的中心频率 $f_0$ 基于网格频散准则

\begin{equation}
dx \le \frac{vp_{min}}{N_\lambda f_0}, \nonumber
\end{equation}

使用哪个有限差分建模运行, 基于Marmousi-2模型定义好的 $dx$ , 最小P波速度 $vp_{min}$ 和 $N_\lambda = 4$ 每个主要波长的网格点.
- 使用3点空间有限差分算子开始Marmousi-2模型的建模运行. 在中心位置放置气枪在 x = 5000 m 和 depth = 40 m 在海平面之下。不要忘了计算合适的时间步长 $dt$
- 添加额外的函数 `update_d2px_d2pz_5pt` 来用二维声学有限差分代码中的5点有限差分算子近似二阶空间导数，推导过程见 [这里](http://nbviewer.jupyter.org/github/daniel-koehn/Theory-of-seismic-waves-II/blob/master/04_FD_stability_dispersion/3_fd_taylor_operators.ipynb). 加入选项 `op` 来切换3点计算和5点计算
- 使用5点算符为Marmousi-2模型启动额外的建模运行。
- 想象你放置了一个 Ocean-Bottom-Cable (OBC) 在海底面上。 通过在x方向的笛卡尔模型的每个网格点上放置接收器，在深度460米处计算一个OBC炮集合。 修改FD代码以记录每个接收器位置的地震图。不要忘记从建模函数返回地震图。
- 分别绘制和比较3点和5点操作产生的地震图。

In [ ]:
# Import Libraries 
# ----------------
import numpy as np
from numba import jit
import matplotlib
import matplotlib.pyplot as plt
from pylab import rcParams

# Ignore Warning Messages
# -----------------------
import warnings
warnings.filterwarnings("ignore")

from mpl_toolkits.axes_grid1 import make_axes_locatable

弹性版本的Marmousi-2模型, 以及高频的有限差分的共炮点道集可参见[here](http://www.agl.uh.edu/downloads/downloads.htm). 空间离散化的Marmousi-2 P波速度模型的中心部分: 

$nx = 500$ gridpoints

$nz = 174$ gridpoints

$dx = dz = 20\; m$

is available as IEEE little endian binary file `marmousi_II_marine.vp` in the `marmousi-2` directory. It can be imported to Python with the following code snippet:

In [ ]:
# Import Marmousi-2 Vp model
# --------------------------

# DEFINE MODEL DISCRETIZATION HERE!
nx = 500       # number of grid points in x-direction
nz = 174       # number of grid points in z-direction
dx = 20        # spatial grid point distance in x-direction (m)
dz = dx        # spatial grid point distance in z-direction (m)

# Define model filename
name_vp = "marmousi-2/marmousi_II_marine.vp"

# Open file and write binary data to vp
f = open(name_vp)
data_type = np.dtype ('float32').newbyteorder ('<')
vp = np.fromfile (f, dtype=data_type)

# Reshape (1 x nx*nz) vector to (nx x nz) matrix 
vp = vp.reshape(nx,nz)

在将模型读入Python之后，我们可以看一下它 ...

In [ ]:
# Plot Marmousi-2 vp-model
# ------------------------

# Define xmax, zmax and model extension
xmax = nx * dx
zmax = nz * dz
extent = [0, xmax, zmax, 0]

fig = plt.figure(figsize=(12,3))  # define figure size

image = plt.imshow((vp.T)/1000, cmap=plt.cm.viridis, interpolation='nearest', 
                   extent=extent)

cbar = plt.colorbar(aspect=10, pad=0.02)
cbar.set_label('Vp [km/s]', labelpad=10)
plt.title('Marmousi-2 model')
plt.xlabel('x [m]')
plt.ylabel('z [m]')
plt.show()

In [ ]:
# Definition of modelling parameters
# ----------------------------------
# DEFINE MAXIMUM RECORDING TIME HERE!
tmax = 6  # maximum wave propagation time (s)

# DEFINE YOUR SHOT POSITION HERE!
xsrc = 5000  # x-source position (m)
zsrc = 40    # z-source position (m)

# DEFINE YOUR RECEIVER POSITION HERE!
# xr = np.linspace(20,10000,500)
# zr = np.ones(500) * 460

xr = 2000
zr = 460

# CALCULATE DOMINANT FREQUENCY OF THE SOURCE WAVELET HERE!
f0   = 15 # dominant frequency of the source (Hz)
print("f0 = ", f0, " Hz")
t0   = 4.0/f0   # source time shift (s)

isnap = 2  # snapshot interval (timesteps)

In [ ]:
@jit(nopython=True) # use JIT for C-performance
def update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz):
    
    for i in range(1, nx - 1):
        for j in range(1, nz - 1):
                
            d2px[i,j] = (p[i + 1,j] - 2 * p[i,j] + p[i - 1,j]) / dx**2                
            d2pz[i,j] = (p[i,j + 1] - 2 * p[i,j] + p[i,j - 1]) / dz**2
        
    return d2px, d2pz 

In [ ]:
# ADD THE SPATIAL 5-POINT OPERATORS HERE!
@jit(nopython=True) # use JIT for C-performance
def update_d2px_d2pz_5pt(p, dx, dz, nx, nz, d2px, d2pz):
    
    for i in range(1, nx - 2):
        for j in range(1, nz - 2):
            
            d2px[i,j] = (p[i + 2,j]*(-1)/12 + p[i + 1,j]*4/3 + p[i,j]*(-5)/2 + p[i - 1,j]*4/3 + p[i - 2, j]*(-1)/12) / dx**2
            d2pz[i,j] = (p[i,j + 2]*(-1)/12 + p[i,j + 1]*4/3 + p[i,j]*(-5)/2 + p[i,j - 1]*4/3 + p[i, j - 2]*(-1)/12) / dx**2
    
        
    return d2px, d2pz 

In [ ]:
# Define simple absorbing boundary frame based on wavefield damping 
# according to Cerjan et al., 1985, Geophysics, 50, 705-708
def absorb(nx,nz):

    FW = 60     # thickness of absorbing frame (gridpoints)    
    a = 0.0053
    
    coeff = np.zeros(FW)
    
    # define coefficients in absorbing frame
    for i in range(FW):    
        coeff[i] = np.exp(-(a**2 * (FW-i)**2))

    # initialize array of absorbing coefficients
    absorb_coeff = np.ones((nx,nz))

    # compute coefficients for left grid boundaries (x-direction)
    zb=0 
    for i in range(FW):
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[i,j] = coeff[i]

    # compute coefficients for right grid boundaries (x-direction)        
    zb=0
    for i in range(FW):
        ii = nx - i - 1
        ze = nz - i - 1
        for j in range(zb,ze):
            absorb_coeff[ii,j] = coeff[i]

    # compute coefficients for bottom grid boundaries (z-direction)        
    xb=0 
    for j in range(FW):
        jj = nz - j - 1
        xb = j
        xe = nx - j
        for i in range(xb,xe):
            absorb_coeff[i,jj] = coeff[j]

    return absorb_coeff

In [ ]:
# FD_2D_acoustic code with JIT optimization
# -----------------------------------------
def FD_2D_acoustic_JIT(vp, dt,dx,dz,f0,xsrc,zsrc,op):        
    
    # calculate number of time steps nt 
    # ---------------------------------
    nt = (int)(tmax/dt)
    
    # locate source on Cartesian FD grid
    # ----------------------------------
    isrc = (int)(xsrc/dx)  # source location in grid in x-direction
    jsrc = (int)(zsrc/dz)  # source location in grid in x-direction    
    
    # locate receiver on Cartesian FD grid
    # ----------------------------------
    ir = (int)(xr/dx)  # receiver location in grid in x-direction
    jr = (int)(zr/dz)  # receiver location in grid in x-direction 
    
    # Source time function (Gaussian)
    # -------------------------------
    src  = np.zeros(nt + 1)
    time = np.linspace(0 * dt, nt * dt, nt)

    # 1st derivative of Gaussian
    src  = -2. * (time - t0) * (f0 ** 2) * (np.exp(- (f0 ** 2) * (time - t0) ** 2)) 
    
    # define clip value: 0.1 * absolute maximum value of source wavelet
    clip = 0.5 * max([np.abs(src.min()), np.abs(src.max())]) / (dx*dz) * dt**2
    
    # Define absorbing boundary frame
    # -------------------------------    
    absorb_coeff = absorb(nx,nz)
    
    # Define squared vp-model
    # -----------------------        
    vp2 = vp**2
    
    # Initialize empty pressure arrays
    # --------------------------------
    p    = np.zeros((nx,nz)) # p at time n (now)
    pold = np.zeros((nx,nz)) # p at time n-1 (past)
    pnew = np.zeros((nx,nz)) # p at time n+1 (present)
    d2px = np.zeros((nx,nz)) # 2nd spatial x-derivative of p
    d2pz = np.zeros((nx,nz)) # 2nd spatial z-derivative of p 
    
    # INITIALIZE SEISMOGRAMS HERE! 
    # ----------------------------
    seis = np.zeros(nt)
        
    # Initalize animation of pressure wavefield 
    # -----------------------------------------    
    fig = plt.figure(figsize=(7,3))  # define figure size
    extent = [0.0,xmax,zmax,0.0]     # define model extension
    
    # Plot Vp-model
    image = plt.imshow((vp.T)/1000, cmap=plt.cm.gray, interpolation='nearest', 
                        extent=extent)    
    
    # Plot pressure wavefield movie
    image1 = plt.imshow(p.T, animated=True, cmap="RdBu", alpha=.75, extent=extent, 
                          interpolation='nearest', vmin=-clip, vmax=clip)    
    plt.title('Pressure wavefield')
    plt.xlabel('x [m]')
    plt.ylabel('z [m]')
           
    plt.ion()    
    plt.show(block=False)
    
    # Calculate Partial Derivatives
    # -----------------------------
    for it in range(nt):
    
        # FD approximation of spatial derivative by 3 point operator
        if(op==3):
            d2px, d2pz = update_d2px_d2pz_3pt(p, dx, dz, nx, nz, d2px, d2pz)
        
        # ADD FD APPROXIMATION OF SPATIAL DERIVATIVES BY 5 POINT OPERATOR HERE!
        if(op==5):
            d2px, d2pz = update_d2px_d2pz_5pt(p, dx, dz, nx, nz, d2px, d2pz)

        # Time Extrapolation
        # ------------------
        pnew = 2 * p - pold + vp2 * dt**2 * (d2px + d2pz)

        # Add Source Term at isrc
        # -----------------------
        # Absolute pressure w.r.t analytical solution
        pnew[isrc,jsrc] = pnew[isrc,jsrc] + src[it] / (dx * dz) * dt ** 2
        
        # Apply absorbing boundary frame
        # ------------------------------
        p *= absorb_coeff
        pnew *= absorb_coeff
        
        # Remap Time Levels
        # -----------------
        pold, p = p, pnew
        
        # WRITE SEISMOGRAMS HERE!
        seis[it] = p[ir,jr]
    
        # display pressure snapshots 
        if (it % isnap) == 0:            
            image1.set_data(p.T)
            fig.canvas.draw()
    
    # DO NOT FORGET TO RETURN THE SEISMOGRAM HERE!
    #if(op==3):
    #    seis_marm_3pt = seis
    #    return seis_marm_3pt
    #if(op==5):
    #    seis_marm_5pt = seis
    #    return seis_marm_5pt

## 模型波在Marmousi-2模型中的传播与二维声学有限差分代码

利用三点算符在Marmouisi-2模型中建模声波传播的时间。我们只需要定义时间步长 $dt$:

In [ ]:
# Run 2D acoustic FD modelling with 3-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 3  # define spatial FD operator (3-point) 

# DEFINE TIME STEP HERE!
dt = 0.001  # time step (s)

FD_2D_acoustic_JIT(vp,dt,dx,dz,f0,xsrc,zsrc,op)

In [ ]:
# Run 2D acoustic FD modelling with 5-point spatial operater
# ----------------------------------------------------------
%matplotlib notebook
op = 5  # define spatial FD operator (5-point) 

# DEFINE TIME STEP HERE!
dt   = 0.001 # time step (s)

FD_2D_acoustic_JIT(vp,dt,dx,dz,f0,xsrc,zsrc,op)

In [ ]:
%matplotlib notebook
# PLOT YOUR MODELLED OBC SHOT GATHER HERE!
clip_seis = 1e-7
extent_seis = [0.0,xmax/1000,tmax,0.0]

plt.subplot(121)
plt.imshow(seis_marm_3pt.T, cmap=plt.cm.gray, aspect=2, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)

plt.title('3-point operator')
plt.xlabel('x [km]')
plt.ylabel('t [s]')


ax = plt.subplot(122)
plt.imshow(seis_marm_5pt.T, cmap=plt.cm.gray, aspect=2, vmin=-clip_seis, 
                   vmax=clip_seis, extent=extent_seis)
ax.set_yticks([]) 
plt.title('5-point operator')
plt.xlabel('x [km]')
#plt.ylabel('t [s]')
plt.tight_layout()
plt.show()

## 我们学习到:

- 如何在复杂的Marmousi-2模型中模拟波的传播。现在你可以建模了 ...